# 02 — Descubrimiento de Señales (Phase 1)

Este notebook explica el **proceso de descubrimiento de señales** del pipeline de trading.
El descubrimiento de señales es la primera fase del pipeline y consiste en evaluar
combinaciones de indicadores técnicos de forma **estadísticamente rigurosa**.

## Metodología

1. Se generan cientos de combinaciones de indicadores con diferentes umbrales
2. Cada combinación se evalúa como señal de entrada en datos históricos (60% training)
3. Se miden métricas clave: Win Rate (WR), Profit Factor (PF), p-value
4. Solo las señales estadísticamente significativas avanzan al siguiente fase

> **Nota**: Los nombres concretos de señales y sus umbrales son confidenciales.
> Este notebook presenta la **metodología**, no las señales específicas.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_theme(style='whitegrid', palette='muted')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['figure.dpi'] = 100

DATA_DIR = Path('../data/sample')

## 1. División cronológica de datos

Para series temporales, la separación de datos **debe ser cronológica** (nunca aleatoria)
para evitar **data leakage temporal** — usar información futura para predecir el pasado.

```
|──── 60% Training ────|──── 20% Validation ────|──── 20% Test ────|
   Señal Discovery           (reservado)             Phase 5
   Optuna (Phase 3)                                  (nunca tocado)
   Training (Phase 4)
```

- **Training (60%)**: Descubrimiento de señales + entrenamiento de modelos
- **Validation (20%)**: Reservado (no usado en pipeline actual)
- **Test (20%)**: Solo se accede en Phase 5 para validación final

In [ ]:
# Cargar datos de rates para demostrar el split
rates = pd.read_csv(DATA_DIR / 'rates' / 'eurusd_h1_rates.csv')
rates['datetime'] = pd.to_datetime(rates['readable_date'])
rates = rates.sort_values('datetime').reset_index(drop=True)

n = len(rates)
train_end = int(n * 0.6)
val_end = int(n * 0.8)

train = rates.iloc[:train_end]
val = rates.iloc[train_end:val_end]
test = rates.iloc[val_end:]

print(f'Total:      {n:,} velas')
print(f'Training:   {len(train):,} velas ({train["datetime"].min()} → {train["datetime"].max()})')
print(f'Validation: {len(val):,} velas ({val["datetime"].min()} → {val["datetime"].max()})')
print(f'Test:       {len(test):,} velas ({test["datetime"].min()} → {test["datetime"].max()})')

# Visualizar split
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(train['datetime'], train['close'], color='steelblue', label=f'Training (60%, {len(train)} velas)')
ax.plot(val['datetime'], val['close'], color='orange', label=f'Validation (20%, {len(val)} velas)')
ax.plot(test['datetime'], test['close'], color='crimson', label=f'Test (20%, {len(test)} velas)')
ax.set_title('División Cronológica de Datos — EURUSD H1', fontsize=13)
ax.set_ylabel('Precio')
ax.legend()
ax.axvline(train['datetime'].max(), color='gray', linestyle='--', alpha=0.5)
ax.axvline(val['datetime'].max(), color='gray', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

## 2. Concepto de señal de trading

Una **señal** es una combinación de condiciones sobre indicadores técnicos que,
cuando se cumplen simultáneamente, sugieren una oportunidad de trading.

### Ejemplo genérico (no es una señal real del sistema)

```
Señal LONG:
  - RSI(14) < umbral_sobreventa
  - Precio > SMA(200)             → tendencia alcista
  - MACD histograma > 0           → momentum positivo
```

### Evaluación con SL/TP fijo

Para evaluar la calidad de una señal de forma estandarizada, se usa un esquema
de **Stop Loss (SL) y Take Profit (TP) simétricos**:

- SL = TP (ratio de riesgo/recompensa 1:1)
- Se incluyen costes de transacción realistas (spread + slippage + comisiones)
- El **Win Rate de breakeven** se calcula considerando estos costes

In [ ]:
# Simulación del concepto: evaluación de señal aleatoria vs breakeven
np.random.seed(42)

# Simular 500 trades con diferentes Win Rates
win_rates = np.arange(0.40, 0.75, 0.01)

def simulate_pf(wr, n_trades=500, sl=30, tp=30, cost=2.5):
    """Simula Profit Factor para un Win Rate dado con costes."""
    net_win = tp - cost   # Ganancia neta por trade ganador
    net_loss = sl + cost  # Pérdida neta por trade perdedor
    wins = int(n_trades * wr)
    losses = n_trades - wins
    gross_profit = wins * net_win
    gross_loss = losses * net_loss
    if gross_loss == 0:
        return float('inf')
    return gross_profit / gross_loss

pfs = [simulate_pf(wr) for wr in win_rates]
breakeven_idx = np.argmin(np.abs(np.array(pfs) - 1.0))

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(win_rates * 100, pfs, color='steelblue', linewidth=2)
ax.axhline(1.0, color='red', linestyle='--', alpha=0.7, label='Breakeven (PF = 1.0)')
ax.axhline(1.2, color='orange', linestyle='--', alpha=0.7, label='Umbral mínimo (PF > 1.2)')
ax.axvline(win_rates[breakeven_idx] * 100, color='red', linestyle=':', alpha=0.5)
ax.fill_between(win_rates * 100, pfs, 1.0, where=np.array(pfs) > 1.0, alpha=0.1, color='green')
ax.fill_between(win_rates * 100, pfs, 1.0, where=np.array(pfs) < 1.0, alpha=0.1, color='red')
ax.set_xlabel('Win Rate (%)', fontsize=12)
ax.set_ylabel('Profit Factor', fontsize=12)
ax.set_title('Profit Factor vs Win Rate (SL/TP simétrico, con costes de transacción)', fontsize=13)
ax.legend()
ax.set_ylim(0.3, 2.5)
plt.tight_layout()
plt.show()

print(f'Win Rate de breakeven (incluyendo costes): ~{win_rates[breakeven_idx]*100:.1f}%')
print(f'PF mínimo exigido: 1.2 (para cubrir costes y varianza estadística)')

## 3. Validación estadística

### El problema del p-hacking

Cuando se evalúan **cientos** de combinaciones, existe el riesgo de encontrar señales
que parecen rentables **por puro azar**. La validación estadística con **p-values**
y la separación estricta de datos (train/test) mitigan este riesgo.

### Distribución bajo hipótesis nula

Si una señal no tiene ventaja real, su Win Rate debería ser cercano al breakeven.
El **p-value** mide la probabilidad de observar un Win Rate tan alto por azar.

In [ ]:
from scipy import stats

# Simulación: distribución de WR bajo hipótesis nula (sin ventaja)
n_simulations = 10000
n_trades = 100
breakeven_wr = 0.54  # WR de breakeven aproximado

# Simular Win Rates bajo H0
null_wrs = np.random.binomial(n_trades, breakeven_wr, n_simulations) / n_trades

fig, ax = plt.subplots(figsize=(12, 5))
ax.hist(null_wrs * 100, bins=40, density=True, color='lightcoral', alpha=0.7,
        edgecolor='white', label='H0: Sin ventaja (WR ≈ breakeven)')

# Marcar una señal hipotética con WR = 62%
observed_wr = 0.62
p_value = np.mean(null_wrs >= observed_wr)
ax.axvline(observed_wr * 100, color='green', linewidth=2, linestyle='--',
           label=f'Señal observada: WR={observed_wr*100:.0f}% (p={p_value:.4f})')

# Marcar zona de rechazo
threshold = np.percentile(null_wrs, 95) * 100
ax.axvline(threshold, color='red', linewidth=1.5, linestyle=':',
           label=f'Umbral p<0.05: WR>{threshold:.1f}%')

ax.set_xlabel('Win Rate (%)', fontsize=12)
ax.set_ylabel('Densidad', fontsize=12)
ax.set_title(f'Test de Hipótesis: ¿La señal tiene ventaja real? (n={n_trades} trades)', fontsize=13)
ax.legend()
plt.tight_layout()
plt.show()

print(f'p-value = {p_value:.4f}')
print(f'Interpretación: {"SIGNIFICATIVA (p < 0.05) — la señal probablemente tiene ventaja real" if p_value < 0.05 else "NO significativa — podría ser azar"}')

## 4. Escala del descubrimiento

El sistema evalúa las combinaciones de forma **automatizada y exhaustiva**:

- 7 familias de indicadores × múltiples periodos = cientos de features
- Combinaciones de 2-3 indicadores con diferentes umbrales
- Evaluadas en 8 pares FOREX × 9 temporalidades
- Cada combinación probada en ambas direcciones (LONG y SHORT)

Este enfoque **sistematizado** contrasta con la selección manual subjetiva
y permite cuantificar la significancia estadística de cada señal.

In [ ]:
# Simulación del proceso de descubrimiento
np.random.seed(123)

n_signals_tested = 200
# Simular señales: mayoría sin ventaja, algunas con ventaja real
true_wrs = np.concatenate([
    np.random.normal(breakeven_wr, 0.04, int(n_signals_tested * 0.85)),  # 85% sin ventaja
    np.random.normal(0.62, 0.03, int(n_signals_tested * 0.15)),          # 15% con ventaja
])
true_wrs = np.clip(true_wrs, 0.3, 0.8)

# Simular trades observados para cada señal
observed_trades = np.random.randint(50, 300, n_signals_tested)
observed_wrs = np.array([np.random.binomial(n, wr) / n for n, wr in zip(observed_trades, true_wrs)])
observed_pfs = np.array([simulate_pf(wr) for wr in observed_wrs])

# Crear DataFrame de resultados
discovery_results = pd.DataFrame({
    'signal_id': range(n_signals_tested),
    'win_rate': observed_wrs,
    'profit_factor': observed_pfs,
    'trades': observed_trades,
    'passed': (observed_pfs > 1.2) & (observed_wrs > breakeven_wr) & (observed_trades >= 50)
})

passed = discovery_results[discovery_results['passed']]
failed = discovery_results[~discovery_results['passed']]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter: WR vs PF
axes[0].scatter(failed['win_rate']*100, failed['profit_factor'],
                alpha=0.4, s=30, color='lightcoral', label=f'No pasan ({len(failed)})')
axes[0].scatter(passed['win_rate']*100, passed['profit_factor'],
                alpha=0.7, s=50, color='green', label=f'Pasan Phase 1 ({len(passed)})')
axes[0].axhline(1.2, color='orange', linestyle='--', alpha=0.5, label='PF mínimo (1.2)')
axes[0].axhline(1.0, color='red', linestyle='--', alpha=0.3)
axes[0].set_xlabel('Win Rate (%)', fontsize=11)
axes[0].set_ylabel('Profit Factor', fontsize=11)
axes[0].set_title('Resultados del Descubrimiento de Señales', fontsize=12)
axes[0].legend()

# Distribución de PF
axes[1].hist(discovery_results['profit_factor'], bins=40, color='steelblue', alpha=0.7, edgecolor='white')
axes[1].axvline(1.0, color='red', linestyle='--', label='Breakeven')
axes[1].axvline(1.2, color='orange', linestyle='--', label='Umbral mínimo')
axes[1].set_xlabel('Profit Factor', fontsize=11)
axes[1].set_ylabel('Frecuencia', fontsize=11)
axes[1].set_title('Distribución de Profit Factor', fontsize=12)
axes[1].legend()

plt.tight_layout()
plt.show()

print(f'Señales evaluadas: {n_signals_tested}')
print(f'Señales que pasan Phase 1: {len(passed)} ({len(passed)/n_signals_tested*100:.1f}%)')

## 5. Conclusiones

### Rigor del proceso

1. **Separación cronológica**: Los datos de test (20%) nunca se tocan durante el descubrimiento
2. **Validación estadística**: Se exige significancia (p-value) para filtrar señales por azar
3. **Costes realistas**: El breakeven incluye spread, slippage y comisiones
4. **Automatización**: Cientos de combinaciones evaluadas sistemáticamente

### Flujo hacia las siguientes fases

Las señales que pasan Phase 1 avanzan a:
- **Phase 3 (Optuna)**: Optimización de hiperparámetros (50 trials)
- **Phase 4 (30-Fold Training)**: Entrenamiento de ensemble de 30 modelos
- **Phase 5 (Test)**: Validación final en datos nunca vistos

> **Siguiente**: `03_rl_training.ipynb` — Entrenamiento por Aprendizaje por Refuerzo